
# GA4 Ecommerce EDA Snapshot

**Dataset:** `bigquery-public-data.ga4_obfuscated_sample_ecommerce`

**Analysis window:** Nov 1–30, 2020

**Engine:** SQL queries executed directly in **Google BigQuery**

## TL;DR

We ran 6 focused EDA queries to check dataset size, funnel coverage, session feasibility, field quality, ecommerce reliability, and traffic-source usability.

✅ 1.47M events, 108K sessions, complete commerce tracking

⚠️ Only 44% usable attribution data

🎯 Focus: Funnel leakage, device/geo segments, session behavior



## What we confirmed

* **Scale is solid:** 1.47M events, 79,421 users, 108,401 sessions, 17 event types (Nov only).
* **Session analysis is valid:** we can reconstruct sessions using
  `session_key = CONCAT(user_pseudo_id, '-', ga_session_id)`.
* **Funnel is measurable:** all core events exist (`view_item`, `add_to_cart`, `begin_checkout`, `purchase`).
* **This is a reach-based funnel:** it shows who touched each stage, not strict step-by-step order.
* **Commerce fields are usable:** transactions + revenue are present and good enough for KPI reporting.
* **Traffic source needs caution:**

  * only **44.33%** users are in clearly usable attributed source/medium values
  * **55.67%** are default or obfuscated buckets
  * source distribution is highly concentrated (top 3 sources = **89.49%**)

## Key interpretation decisions

* Use **transaction-level** counts for finance metrics (not raw purchase event count).
* Exclude `shipping_value` from KPI logic in this slice (100% null on purchase rows).
* Treat `traffic_source.source/medium` as **secondary context** (first-touch, coarse channel cuts).
* Keep an explicit **Unknown/Obfuscated** channel bucket in any source/medium report.
* Do not make deep attribution claims from this sample.

## What this enables next

We can confidently move into:

1. Funnel leakage analysis (especially product view → cart gap)
2. Device/geo segmentation of conversion + revenue
3. Session-level behavior patterns tied to conversion outcomes

And we should keep attribution analysis as a limited, caveated layer — not the core story.


In [ ]:
-- Query 1: Big picture (Nov 2020 sample month)
WITH base AS (
  SELECT
    event_date,
    PARSE_DATE('%Y%m%d', event_date) AS event_dt,
    event_name,
    user_pseudo_id,
    (SELECT value.int_value FROM UNNEST(event_params) WHERE key = 'ga_session_id') AS ga_session_id,
    ecommerce.transaction_id AS transaction_id,
    ecommerce.purchase_revenue AS purchase_revenue
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20201130'
),
agg AS (
  SELECT
    COUNT(*) AS total_events,
    COUNT(DISTINCT user_pseudo_id) AS total_users,
    COUNT(DISTINCT event_date) AS total_days,
    COUNT(DISTINCT event_name) AS unique_event_types,
    MIN(event_dt) AS first_date,
    MAX(event_dt) AS last_date,

    COUNT(DISTINCT IF(
      ga_session_id IS NOT NULL,
      CONCAT(user_pseudo_id, '-', CAST(ga_session_id AS STRING)),
      NULL
    )) AS sessions,

    COUNTIF(event_name = 'page_view') AS pageviews,
    COUNTIF(event_name = 'view_item') AS product_views,
    COUNTIF(event_name = 'add_to_cart') AS add_to_carts,
    COUNTIF(event_name = 'begin_checkout') AS checkouts,
    COUNTIF(event_name = 'purchase') AS purchase_events,

    COUNT(DISTINCT IF(
      event_name = 'purchase' AND ga_session_id IS NOT NULL,
      CONCAT(user_pseudo_id, '-', CAST(ga_session_id AS STRING)),
      NULL
    )) AS purchase_sessions,

    COUNT(DISTINCT IF(event_name = 'purchase', transaction_id, NULL)) AS transactions,
    SUM(IF(event_name = 'purchase', purchase_revenue, 0)) AS total_revenue
  FROM base
)

SELECT
  total_events,
  total_users,
  total_days,
  unique_event_types,
  first_date,
  last_date,
  sessions,
  pageviews,
  product_views,
  add_to_carts,
  checkouts,
  purchase_events,
  transactions,
  ROUND(total_revenue, 2) AS total_revenue,

  ROUND(SAFE_DIVIDE(total_events, total_users), 2) AS events_per_user,
  ROUND(SAFE_DIVIDE(pageviews, sessions), 2) AS pages_per_session,
  ROUND(100 * SAFE_DIVIDE(purchase_sessions, sessions), 2) AS session_conversion_rate_pct,
  ROUND(SAFE_DIVIDE(total_revenue, transactions), 2) AS avg_order_value
FROM agg;



## Query 1 — Executive Summary (Nov 2020)

### Objective

Quick health check of dataset scale, session feasibility, funnel coverage, and revenue signal.


### Session Logic

GA4 export is event-level, so sessions must be reconstructed.

| Step | Field                                                      | Meaning                             |
| ---- | ---------------------------------------------------------- | ----------------------------------- |
| 1    | `ga_session_id` (from `event_params`)                      | Session identifier generated by GA4 |
| 2    | `user_pseudo_id`                                           | User/browser-level identifier       |
| 3    | `session_key = CONCAT(user_pseudo_id, '-', ga_session_id)` | Unique key for one user-session     |

**Why this works:** `ga_session_id` can repeat across different users, so it is not unique by itself. Pairing it with `user_pseudo_id` makes it session-unique.


### Output (02/08 run)

| Metric                      |                   Value |
| --------------------------- | ----------------------: |
| Total events                |               1,472,712 |
| Total users                 |                  79,421 |
| Total days                  |                      30 |
| Date range                  | 2020-11-01 → 2020-11-30 |
| Unique event types          |                      17 |
| Sessions (reconstructed)    |                 108,401 |
| Pageviews                   |                 453,904 |
| Product views (`view_item`) |                 148,639 |
| Add to carts                |                   7,674 |
| Begin checkouts             |                   9,546 |
| Purchase events             |                   2,054 |
| Transactions (distinct IDs) |                   1,259 |
| Total revenue               |              144,260.00 |
| Events per user             |                   18.54 |
| Pages per session           |                    4.19 |
| Session conversion rate     |                   1.49% |
| Average order value         |                  114.58 |


### Analyst Readout

| Observation                        | What it means                                       | Decision impact                                 |
| ---------------------------------- | --------------------------------------------------- | ----------------------------------------------- |
| 1.47M events / 79K users           | Strong volume for stable funnel analysis            | Proceed with funnel + segmentation              |
| 108K reconstructed sessions        | Sessionization is viable                            | Session-level KPIs are valid                    |
| CVR = 1.49%                        | Realistic ecommerce conversion range                | Good baseline for improvement scenarios         |
| Revenue + transaction IDs present  | Commerce tracking is usable                         | Can report AOV, conversion, revenue per segment |
| `begin_checkout` > `add_to_cart`   | Funnel instrumentation likely non-linear/misaligned | Validate event definitions before final story   |
| `purchase_events` > `transactions` | Possible duplicate purchase events or repeat firing | Use **transactions** for financial reporting    |



In [ ]:
-- Query 2: Funnel by users + sessions (clean denominators)
WITH raw AS (
  SELECT
    event_name,
    user_pseudo_id,
    (SELECT value.int_value FROM UNNEST(event_params) WHERE key = 'ga_session_id') AS ga_session_id
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20201130'
),
base AS (
  SELECT
    event_name,
    user_pseudo_id,
    IF(
      ga_session_id IS NULL,
      NULL,
      CONCAT(user_pseudo_id, '-', CAST(ga_session_id AS STRING))
    ) AS session_key
  FROM raw
),
totals AS (
  SELECT
    COUNT(DISTINCT user_pseudo_id) AS total_users,
    COUNT(DISTINCT session_key) AS total_sessions
  FROM base
  WHERE session_key IS NOT NULL
),
steps AS (
  SELECT
    event_name,
    COUNT(*) AS event_count,
    COUNT(DISTINCT user_pseudo_id) AS users_reached,
    COUNT(DISTINCT session_key) AS sessions_reached
  FROM base
  WHERE event_name IN (
    'session_start', 'page_view', 'view_item',
    'add_to_cart', 'begin_checkout', 'purchase'
  )
  GROUP BY event_name
)
SELECT
  s.event_name,
  s.event_count,
  s.users_reached,
  s.sessions_reached,
  ROUND(100 * SAFE_DIVIDE(s.users_reached, t.total_users), 2) AS pct_of_users,
  ROUND(100 * SAFE_DIVIDE(s.sessions_reached, t.total_sessions), 2) AS pct_of_sessions
FROM steps s
CROSS JOIN totals t
ORDER BY CASE s.event_name
  WHEN 'session_start' THEN 1
  WHEN 'page_view' THEN 2
  WHEN 'view_item' THEN 3
  WHEN 'add_to_cart' THEN 4
  WHEN 'begin_checkout' THEN 5
  WHEN 'purchase' THEN 6
  ELSE 99
END;


## Query 2 — Funnel Coverage & Drop-off Check

### Objective

Evaluate how much traffic reaches each core ecommerce step and where the biggest volume loss happens.

### What this query does

This is a **reach-based funnel analysis** (not strict sequential pathing). That means it's:

* filters to key funnel events
  `session_start → page_view → view_item → add_to_cart → begin_checkout → purchase`
* reports step-level volume using:

  * `event_count` (how many times event fired)
  * `users_reached` (distinct users touching that step)
  * `sessions_reached` (distinct sessions touching that step)
* normalizes each step to total user/session base:

  * `% of users`
  * `% of sessions`

So this measures **step penetration** (who touched each stage), not guaranteed stage-by-stage order.


### Output

| Step           | Event Count | Users Reached | Sessions Reached | % of Users | % of Sessions |
| -------------- | ----------: | ------------: | ---------------: | ---------: | ------------: |
| session_start  |     106,585 |        78,383 |          106,550 |      98.69 |         98.29 |
| page_view      |     453,904 |        79,181 |           98,555 |      99.70 |         90.92 |
| view_item      |     148,639 |        21,440 |           25,943 |      27.00 |         23.93 |
| add_to_cart    |       7,674 |         2,060 |            2,201 |       2.59 |          2.03 |
| begin_checkout |       9,546 |         4,219 |            4,585 |       5.31 |          4.23 |
| purchase       |       2,054 |         1,532 |            1,617 |       1.93 |          1.49 |


### Analyst notes

* Funnel coverage is usable: all major commerce events are present.
* Biggest leakage is still **view_item → add_to_cart** (23.93% sessions reach `view_item`, only 2.03% reach `add_to_cart`).
* `begin_checkout` > `add_to_cart` (4.23% vs 2.03% sessions) indicates either:

  * tracking/instrumentation mismatch, or
  * alternative behavior patterns (e.g., direct checkout paths).
* Purchase reach is **1.49% of sessions** and **1.93% of users**, which is realistic for ecommerce.


### Important interpretation 

Because this is reach-based:

* a user/session can appear in multiple steps without following strict order,
* users may skip stages (e.g., direct-to-product/direct-to-checkout behavior),
* repeated behaviors (view → search → view, etc.) inflate event counts without changing reach counts.



In [ ]:
WITH base AS (
  SELECT
    event_date,
    user_pseudo_id,
    event_name,
    platform,
    event_timestamp,

    device.category AS device_category,
    geo.country AS geo_country,
    geo.region AS geo_region,
    geo.city AS geo_city,

    traffic_source.source AS ts_source,
    traffic_source.medium AS ts_medium,
    traffic_source.name AS ts_campaign,

    (SELECT value.int_value
     FROM UNNEST(event_params)
     WHERE CAST(key AS STRING) = 'ga_session_id') AS ga_session_id,

    (SELECT value.string_value
     FROM UNNEST(event_params)
     WHERE CAST(key AS STRING) = 'source') AS ep_source,

    (SELECT value.string_value
     FROM UNNEST(event_params)
     WHERE CAST(key AS STRING) = 'medium') AS ep_medium,

    (SELECT value.string_value
     FROM UNNEST(event_params)
     WHERE CAST(key AS STRING) = 'campaign') AS ep_campaign
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20201130'
)
SELECT
  COUNT(*) AS total_rows,

  ROUND(100 * SAFE_DIVIDE(COUNTIF(user_pseudo_id IS NULL), COUNT(*)), 2) AS pct_null_user_pseudo_id,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(event_name IS NULL), COUNT(*)), 2) AS pct_null_event_name,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(event_timestamp IS NULL), COUNT(*)), 2) AS pct_null_event_timestamp,

  ROUND(100 * SAFE_DIVIDE(COUNTIF(ga_session_id IS NULL), COUNT(*)), 2) AS pct_null_ga_session_id,

  ROUND(100 * SAFE_DIVIDE(COUNTIF(device_category IS NULL), COUNT(*)), 2) AS pct_null_device_category,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(geo_country IS NULL), COUNT(*)), 2) AS pct_null_geo_country,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(geo_region IS NULL), COUNT(*)), 2) AS pct_null_geo_region,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(geo_city IS NULL), COUNT(*)), 2) AS pct_null_geo_city,

  ROUND(100 * SAFE_DIVIDE(COUNTIF(ts_source IS NULL OR ts_source = ''), COUNT(*)), 2) AS pct_null_or_blank_traffic_source_source,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(ts_medium IS NULL OR ts_medium = ''), COUNT(*)), 2) AS pct_null_or_blank_traffic_source_medium,

  ROUND(100 * SAFE_DIVIDE(COUNTIF(ep_source IS NULL OR ep_source = ''), COUNT(*)), 2) AS pct_null_or_blank_event_param_source,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(ep_medium IS NULL OR ep_medium = ''), COUNT(*)), 2) AS pct_null_or_blank_event_param_medium
FROM base;



## Query 3 - Check Null for core fields
### Objective

Document what is confirmed so far from null-rate checks before moving deeper into attribution/session modeling.
## 1) Core Null Audit

### Output

| Metric                                  |     Value |
| --------------------------------------- | --------: |
| total_rows                              | 1,472,712 |
| pct_null_user_pseudo_id                 |     0.00% |
| pct_null_event_name                     |     0.00% |
| pct_null_event_timestamp                |     0.00% |
| pct_null_ga_session_id                  |     0.00% |
| pct_null_device_category                |     0.00% |
| pct_null_geo_country                    |     0.00% |
| pct_null_geo_region                     |     0.00% |
| pct_null_geo_city                       |     0.00% |
| pct_null_or_blank_traffic_source_source |     0.00% |
| pct_null_or_blank_traffic_source_medium |     0.00% |
| pct_null_or_blank_event_param_source    |    66.42% |
| pct_null_or_blank_event_param_medium    |    66.03% |

### Analyst notes

* Core structural fields are stable (`user_pseudo_id`, `event_name`, `event_timestamp` all fully populated).
* Session reconstruction is feasible (`ga_session_id` null rate = 0% in this slice).
* Device and geo fields are fully populated in this audit output.
* Traffic-source interpretation needs care:

  * `traffic_source.source/medium` has 0% null/blank, **but this only confirms presence**, not quality or scope fitness.
  * `event_params` source/medium is missing on ~66% of rows, which is expected for event-level params across all events (not every event carries acquisition params).



In [ ]:
WITH purchases AS (
  SELECT
    ecommerce.transaction_id AS transaction_id,
    ecommerce.purchase_revenue AS purchase_revenue,
    ecommerce.purchase_revenue_in_usd AS purchase_revenue_usd,
    ecommerce.shipping_value AS shipping_value,
    ecommerce.tax_value AS tax_value,
    ecommerce.total_item_quantity AS total_item_quantity
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20201130'
    AND event_name = 'purchase'
)
SELECT
  COUNT(*) AS purchase_events,
  COUNT(DISTINCT transaction_id) AS distinct_transactions,

  ROUND(100 * SAFE_DIVIDE(COUNTIF(transaction_id IS NULL OR transaction_id = ''), COUNT(*)), 2) AS pct_null_txn_id,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(purchase_revenue IS NULL), COUNT(*)), 2) AS pct_null_purchase_revenue,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(purchase_revenue_usd IS NULL), COUNT(*)), 2) AS pct_null_purchase_revenue_usd,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(shipping_value IS NULL), COUNT(*)), 2) AS pct_null_shipping_value,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(tax_value IS NULL), COUNT(*)), 2) AS pct_null_tax_value,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(total_item_quantity IS NULL), COUNT(*)), 2) AS pct_null_total_item_quantity
FROM purchases;




## Query 4 - Ecommerce Null Audit 

### Output

| Metric                        |   Value |
| ----------------------------- | ------: |
| purchase_events               |   2,054 |
| distinct_transactions         |   1,259 |
| pct_null_txn_id               |   1.12% |
| pct_null_purchase_revenue     |   2.73% |
| pct_null_purchase_revenue_usd |   0.00% |
| pct_null_shipping_value       | 100.00% |
| pct_null_tax_value            |   2.73% |
| pct_null_total_item_quantity  |   2.73% |

### Analyst notes

* Purchase tracking is usable: transaction and revenue fields are largely present.
* `purchase_revenue_in_usd` is fully populated on purchase rows in this slice.
* `shipping_value` is fully null (100%) and should be excluded from KPI logic.
* `purchase_events` > `distinct_transactions` confirms event count and transaction count should not be used interchangeably.
* Finance-facing metrics should prefer transaction-level aggregation where possible.

In [ ]:
WITH user_ft AS (
  SELECT
    user_pseudo_id,
    LOWER(TRIM(ANY_VALUE(traffic_source.source))) AS source,
    LOWER(TRIM(ANY_VALUE(traffic_source.medium))) AS medium
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20201130'
  GROUP BY user_pseudo_id
),

classified AS (
  SELECT
    user_pseudo_id,
    source,
    medium,
    CASE
      WHEN source IS NULL OR source = '' OR medium IS NULL OR medium = '' THEN 'null_or_blank'
      WHEN source IN ('<other>', '(data deleted)', '(not set)', 'not set', 'unknown', 'unassigned', '(unassigned)')
        OR medium IN ('<other>', '(data deleted)', '(not set)', 'not set', 'unknown', 'unassigned', '(unassigned)')
        THEN 'placeholder_or_deleted'
      WHEN source = '(direct)' OR medium IN ('(none)', '(not set)', 'none', 'not set')
        THEN 'default_unattributed'
      ELSE 'usable_attributed'
    END AS quality_bucket
  FROM user_ft
),

source_rank AS (
  SELECT
    source,
    COUNT(*) AS users,
    100 * SAFE_DIVIDE(COUNT(*), SUM(COUNT(*)) OVER()) AS pct,
    ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS rn
  FROM classified
  GROUP BY source
),

source_medium_rank AS (
  SELECT
    source,
    medium,
    COUNT(*) AS users,
    100 * SAFE_DIVIDE(COUNT(*), SUM(COUNT(*)) OVER()) AS pct,
    ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS rn
  FROM classified
  GROUP BY source, medium
)

SELECT
  COUNT(*) AS total_users,

  COUNTIF(quality_bucket = 'usable_attributed') AS usable_users,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(quality_bucket = 'usable_attributed'), COUNT(*)), 2) AS usable_pct,

  COUNTIF(quality_bucket = 'default_unattributed') AS default_users,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(quality_bucket = 'default_unattributed'), COUNT(*)), 2) AS default_pct,

  COUNTIF(quality_bucket = 'placeholder_or_deleted') AS placeholder_users,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(quality_bucket = 'placeholder_or_deleted'), COUNT(*)), 2) AS placeholder_pct,

  COUNTIF(quality_bucket = 'null_or_blank') AS null_blank_users,
  ROUND(100 * SAFE_DIVIDE(COUNTIF(quality_bucket = 'null_or_blank'), COUNT(*)), 2) AS null_blank_pct,

  (SELECT ROUND(MAX(pct), 2) FROM source_rank WHERE rn = 1) AS top1_source_pct,
  (SELECT ROUND(SUM(pct), 2) FROM source_rank WHERE rn <= 3) AS top3_source_pct,
  (SELECT ROUND(SUM(POW(pct / 100.0, 2)), 4) FROM source_rank) AS source_hhi,

  (SELECT ROUND(MAX(pct), 2) FROM source_medium_rank WHERE rn = 1) AS top1_source_medium_pct,
  (SELECT ROUND(SUM(pct), 2) FROM source_medium_rank WHERE rn <= 5) AS top5_source_medium_pct,
  (SELECT ROUND(SUM(POW(pct / 100.0, 2)), 4) FROM source_medium_rank) AS source_medium_hhi
FROM classified;



## Query 5: Source/Medium Quality + Concentration

### What this query is for

- Step 1: Classify each user by traffic source quality
- Step 2: Count how many users fall into each bucket
- Step 3: Measure concentration (how much traffic is dominated by a few sources)
- Step 4: Make decision manually (use / use with caution / drop)

### Output

| Metric                     |           Value |
| -------------------------- | --------------: |
| Total users                |          79,421 |
| Usable users               | 35,211 (44.33%) |
| Default unattributed users | 18,614 (23.44%) |
| Placeholder/deleted users  | 25,596 (32.23%) |
| Null/blank users           |       0 (0.00%) |
| Top 1 source share         |          37.49% |
| Top 3 source share         |          89.49% |
| Source HHI                 |          0.2831 |
| Top 1 source/medium share  |          32.83% |
| Top 5 source/medium share  |          88.55% |
| Source/medium HHI          |          0.2064 |

### Quick read

* This field is populated, but quality is mixed.
* Only **44.33%** of users are in clearly usable attributed values.
* More than half of users are either default unattributed or obfuscated.
* Distribution is very concentrated (top 3 sources already ~89.5%).

### Practical decision

Use `traffic_source.source/medium` as **first-touch context only**, not as a core attribution driver.




In [ ]:
WITH user_ft AS (
  SELECT
    user_pseudo_id,
    LOWER(TRIM(ANY_VALUE(traffic_source.source))) AS source,
    LOWER(TRIM(ANY_VALUE(traffic_source.medium))) AS medium
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE _TABLE_SUFFIX BETWEEN '20201101' AND '20201130'
  GROUP BY user_pseudo_id
),

classified AS (
  SELECT
    source,
    medium,
    CASE
      WHEN source IS NULL OR source = '' OR medium IS NULL OR medium = '' THEN 'null_or_blank'
      WHEN source IN ('<other>', '(data deleted)', '(not set)', 'not set', 'unknown', 'unassigned', '(unassigned)')
        OR medium IN ('<other>', '(data deleted)', '(not set)', 'not set', 'unknown', 'unassigned', '(unassigned)')
        THEN 'placeholder_or_deleted'
      WHEN source = '(direct)' OR medium IN ('(none)', '(not set)', 'none', 'not set')
        THEN 'default_unattributed'
      ELSE 'usable_attributed'
    END AS quality_bucket
  FROM user_ft
),

dist AS (
  SELECT
    source,
    medium,
    quality_bucket,
    COUNT(*) AS users
  FROM classified
  GROUP BY source, medium, quality_bucket
)

SELECT
  ROW_NUMBER() OVER (ORDER BY users DESC) AS rn,
  source,
  medium,
  quality_bucket,
  users,
  ROUND(100 * SAFE_DIVIDE(users, SUM(users) OVER()), 2) AS pct_users,
  ROUND(
    100 * SAFE_DIVIDE(
      SUM(users) OVER (ORDER BY users DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW),
      SUM(users) OVER()
    ),
    2
  ) AS cumulative_pct
FROM dist
ORDER BY users DESC
LIMIT 30;


## Query 6: Top source/medium distribution table

### What this query is for

Show the actual top source/medium combos so we can see what is real signal vs default/obfuscated traffic.

### Output highlights

* `google / organic` = **32.74%** (largest clean bucket)
* `(direct) / (none)` = **23.38%** (expected default bucket)
* Placeholder/deleted combinations (`<other>`, `(data deleted)`) occupy multiple top ranks
* Top 5 combinations already cover **88.50%** of users

### Quick read

* There is real signal (google organic, google cpc, referral), so the field is not useless.
* But obfuscation/default buckets are too large for deep channel conclusions.
* Keep channel cuts coarse and always include an **Unknown/Obfuscated** bucket in reporting.
